<a href="https://colab.research.google.com/github/Tar-ive/grants_recsys/blob/master/raw_data_sources/nsf_funding.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

#getting NSF funding data from API

In [ ]:
import requests
import pandas as pd
from datetime import datetime, timedelta
import time
from tqdm.notebook import tqdm

class NSFGrantsAPI:
    def __init__(self):
        self.base_url = "https://api.nsf.gov/services/v1/awards.json"

    def fetch_grants(self, offset=1, rpp=25):
        """
        Fetches grants from NSF API with pagination
        """
        # Get dates for last 2 years
        end_date = datetime.now()
        start_date = end_date - timedelta(days=730)  # ~2 years

        params = {
            'dateStart': start_date.strftime("%m/%d/%Y"),
            'dateEnd': end_date.strftime("%m/%d/%Y"),
            'offset': offset,
            'rpp': rpp,
            'printFields': (
                'id,title,agency,awardeeName,fundsObligatedAmt,'
                'startDate,expDate,abstractText,piFirstName,piLastName,'
                'awardeeStateCode,cfdaNumber,date,estimatedTotalAmt'
            )
        }

        try:
            print(f"Making request with params: {params}")
            response = requests.get(self.base_url, params=params)
            response.raise_for_status()

            data = response.json()

            # Check for error response
            if 'serviceNotification' in data.get('response', {}):
                error_msg = data['response']['serviceNotification'][0]['notificationMessage']
                print(f"API Error: {error_msg}")
                return None

            return data

        except requests.exceptions.RequestException as e:
            print(f"Error fetching data from NSF API: {e}")
            if hasattr(e, 'response') and e.response is not None:
                print(f"Response content: {e.response.content}")
            return None

    def transform_grant_data(self, award):
        """
        Transforms NSF award data to match our schema
        """
        try:
            # Extract PI name
            pi_name = f"{award.get('piFirstName', '')} {award.get('piLastName', '')}".strip()

            return {
                'opportunity_id': award.get('id'),
                'opportunity_title': award.get('title'),
                'agency': 'NSF',
                'agency_name': award.get('agency'),
                'award_amount': float(award.get('fundsObligatedAmt', 0)),
                'post_date': award.get('startDate') or award.get('date'),
                'close_date': award.get('expDate'),
                'summary_description': award.get('abstractText', ''),
                'applicant_types': award.get('awardeeName', ''),
                'estimated_total': float(award.get('estimatedTotalAmt', 0)),
                'pi_name': pi_name,
                'cfda_number': award.get('cfdaNumber'),
                'state': award.get('awardeeStateCode')
            }
        except Exception as e:
            print(f"Error transforming award data: {e}")
            print(f"Problematic award data: {award}")
            return None

    def get_all_grants(self, max_records=5000):
        """
        Fetches all grants up to max_records
        """
        all_grants = []
        offset = 1
        rpp = 25  # Maximum allowed by API

        with tqdm(total=max_records) as pbar:
            while len(all_grants) < max_records:
                print(f"\nFetching grants {offset} to {offset + rpp - 1}...")

                response_data = self.fetch_grants(offset=offset, rpp=rpp)

                if not response_data or 'response' not in response_data:
                    print("No response data received")
                    break

                awards = response_data['response'].get('award', [])
                if not awards:
                    print("No awards in response")
                    break

                # Transform each award
                transformed_awards = []
                for award in awards:
                    transformed = self.transform_grant_data(award)
                    if transformed:
                        transformed_awards.append(transformed)

                all_grants.extend(transformed_awards)

                offset += rpp
                pbar.update(len(transformed_awards))

                # Respect rate limits
                time.sleep(0.5)  # 500ms delay between requests

                if len(awards) < rpp:  # Last page
                    break

        print(f"\nTotal grants fetched: {len(all_grants)}")
        return pd.DataFrame(all_grants)

def fetch_and_process_nsf_grants(max_records=5000):
    """
    Main function to fetch and process NSF grants
    """
    api = NSFGrantsAPI()

    # Fetch grants
    print("Fetching grants from NSF API...")
    grants_df = api.get_all_grants(max_records=max_records)

    if grants_df.empty:
        print("No grants data retrieved")
        return grants_df

    # Drop rows with missing critical values
    cleaned_df = grants_df.dropna(subset=['post_date', 'close_date', 'award_amount', 'summary_description'])

    # Convert dates to datetime
    cleaned_df['post_date'] = pd.to_datetime(cleaned_df['post_date'])
    cleaned_df['close_date'] = pd.to_datetime(cleaned_df['close_date'])

    print("\nCleaning Summary:")
    print(f"Original records: {len(grants_df)}")
    print(f"Cleaned records: {len(cleaned_df)}")
    print("\nColumns with null values:")
    print(cleaned_df.isnull().sum())

    return cleaned_df

# Run the code
if __name__ == "__main__":
    try:
        nsf_grants_df = fetch_and_process_nsf_grants()
        if not nsf_grants_df.empty:
            nsf_grants_df.to_csv('nsf_grants.csv', index=False)
            print("\nSample of fetched data:")
            print(nsf_grants_df.head())
    except Exception as e:
        print(f"Error in main execution: {e}")

Fetching grants from NSF API...


  0%|          | 0/5000 [00:00<?, ?it/s]


Fetching grants 1 to 25...
Making request with params: {'dateStart': '01/29/2023', 'dateEnd': '01/28/2025', 'offset': 1, 'rpp': 25, 'printFields': 'id,title,agency,awardeeName,fundsObligatedAmt,startDate,expDate,abstractText,piFirstName,piLastName,awardeeStateCode,cfdaNumber,date,estimatedTotalAmt'}

Fetching grants 26 to 50...
Making request with params: {'dateStart': '01/29/2023', 'dateEnd': '01/28/2025', 'offset': 26, 'rpp': 25, 'printFields': 'id,title,agency,awardeeName,fundsObligatedAmt,startDate,expDate,abstractText,piFirstName,piLastName,awardeeStateCode,cfdaNumber,date,estimatedTotalAmt'}

Fetching grants 51 to 75...
Making request with params: {'dateStart': '01/29/2023', 'dateEnd': '01/28/2025', 'offset': 51, 'rpp': 25, 'printFields': 'id,title,agency,awardeeName,fundsObligatedAmt,startDate,expDate,abstractText,piFirstName,piLastName,awardeeStateCode,cfdaNumber,date,estimatedTotalAmt'}

Fetching grants 76 to 100...
Making request with params: {'dateStart': '01/29/2023', 'dat

#getting keywords, embedding and storing in db.

In [ ]:
import pandas as pd
import numpy as np
from sentence_transformers import SentenceTransformer
from sklearn.feature_extraction.text import CountVectorizer
import torch
import psycopg2
from tqdm.notebook import tqdm
import re
from google.colab import userdata  # For secrets

DB_URL = userdata.get('DATABASE_URL')

# Run the upload process
main()

def preprocess_text(text):
    """Preprocesses text for keyword extraction"""
    if pd.isna(text):
        return ""

    text = str(text)
    text = re.sub(r'[^\w\s]', ' ', text)
    text = re.sub(r'\d+', ' ', text)
    text = text.lower().strip()
    text = re.sub(r'\s+', ' ', text)

    return text

def extract_keywords(text, model, vectorizer, n_keywords=5):
    """Extracts keywords using BERT embeddings"""
    if not text or pd.isna(text):
        return []

    text = preprocess_text(text)

    try:
        candidates = vectorizer.get_feature_names_out()

        # Using GPU for embeddings
        with torch.cuda.device(0):
            text_embedding = model.encode([text], convert_to_tensor=True)
            candidate_embeddings = model.encode(candidates, convert_to_tensor=True)

        # Move to CPU for numpy operations
        similarities = torch.mm(candidate_embeddings, text_embedding.T).flatten().cpu().numpy()

        top_idx = similarities.argsort()[-n_keywords:][::-1]
        keywords = [candidates[i] for i in top_idx]

        return keywords
    except Exception as e:
        print(f"Error processing text: {e}")
        return []

def generate_embeddings(texts, model, batch_size=32):
    """Generate embeddings for a list of texts"""
    embeddings = []

    for i in tqdm(range(0, len(texts), batch_size), desc="Generating embeddings"):
        batch_texts = texts[i:i + batch_size]
        with torch.cuda.device(0):
            batch_embeddings = model.encode(batch_texts, convert_to_tensor=True)
            embeddings.extend(batch_embeddings.cpu().numpy())

    return np.array(embeddings)

def process_grants_with_embeddings(grants_df, model_name='all-MiniLM-L6-v2', batch_size=32):
    """Process grants data with keywords and embeddings"""
    print("Initializing model...")
    device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
    print(f"Using device: {device}")

    model = SentenceTransformer(model_name)
    model = model.to(device)

    print("\nInitializing vectorizer...")
    vectorizer = CountVectorizer(
        ngram_range=(1, 2),
        stop_words='english',
        max_features=1000
    )

    # Fit vectorizer on all summaries
    all_texts = grants_df['summary_description'].astype(str).tolist()
    vectorizer.fit(all_texts)

    # Generate keywords
    print("\nGenerating keywords...")
    keywords_list = []
    for i in tqdm(range(0, len(grants_df), batch_size), desc="Extracting keywords"):
        batch = grants_df.iloc[i:i + batch_size]
        batch_keywords = [
            extract_keywords(text, model, vectorizer)
            for text in batch['summary_description']
        ]
        keywords_list.extend(batch_keywords)

    grants_df['keywords'] = keywords_list

    # Generate embeddings
    print("\nGenerating embeddings for summaries...")
    summary_embeddings = generate_embeddings(all_texts, model, batch_size)

    print("Generating embeddings for keywords...")
    keyword_texts = [' '.join(keywords) for keywords in keywords_list]
    keyword_embeddings = generate_embeddings(keyword_texts, model, batch_size)

    # Add embeddings to dataframe
    grants_df['summary_embedding'] = list(summary_embeddings)
    grants_df['keyword_embedding'] = list(keyword_embeddings)

    return grants_df, summary_embeddings, keyword_embeddings

def save_to_database(df, summary_embeddings, keyword_embeddings, db_url):
    """Save processed grants to database"""
    print("\nSaving to database...")

    try:
        conn = psycopg2.connect(db_url)
        with conn.cursor() as cur:
            # Create vector extension if it doesn't exist
            cur.execute("CREATE EXTENSION IF NOT EXISTS vector;")

            # Create table if it doesn't exist
            create_table_query = """
            CREATE TABLE IF NOT EXISTS nsf_grants (
                id SERIAL PRIMARY KEY,
                opportunity_id TEXT,
                opportunity_title TEXT,
                agency TEXT,
                agency_name TEXT,
                award_amount NUMERIC,
                post_date DATE,
                close_date DATE,
                summary_description TEXT,
                applicant_types TEXT,
                estimated_total NUMERIC,
                pi_name TEXT,
                cfda_number TEXT,
                state TEXT,
                keywords TEXT[],
                summary_embedding vector(384),
                keyword_embedding vector(384),
                created_at TIMESTAMP WITH TIME ZONE DEFAULT CURRENT_TIMESTAMP
            );
            """
            cur.execute(create_table_query)

            # Create indexes
            cur.execute("CREATE INDEX IF NOT EXISTS idx_nsf_grants_opportunity_id ON nsf_grants(opportunity_id);")
            cur.execute("CREATE INDEX IF NOT EXISTS idx_nsf_grants_post_date ON nsf_grants(post_date);")

            # Insert data
            for i in tqdm(range(len(df)), desc="Inserting records"):
                row = df.iloc[i]

                insert_query = """
                INSERT INTO nsf_grants (
                    opportunity_id, opportunity_title, agency, agency_name,
                    award_amount, post_date, close_date, summary_description,
                    applicant_types, estimated_total, pi_name, cfda_number,
                    state, keywords, summary_embedding, keyword_embedding
                ) VALUES (%s, %s, %s, %s, %s, %s, %s, %s, %s, %s, %s, %s, %s, %s, %s, %s)
                """

                cur.execute(insert_query, (
                    str(row['opportunity_id']),
                    str(row['opportunity_title']),
                    str(row['agency']),
                    str(row['agency_name']),
                    float(row['award_amount']),
                    row['post_date'],
                    row['close_date'],
                    str(row['summary_description']),
                    str(row['applicant_types']),
                    float(row['estimated_total']),
                    str(row['pi_name']),
                    str(row['cfda_number']),
                    str(row['state']),
                    list(row['keywords']),
                    summary_embeddings[i].tolist(),
                    keyword_embeddings[i].tolist()
                ))

                if (i + 1) % 100 == 0:
                    conn.commit()

            conn.commit()
            print("\nData saved successfully!")

    except Exception as e:
        print(f"Error saving to database: {e}")
        if conn:
            conn.rollback()
    finally:
        if conn:
            conn.close()

def process_and_save_nsf_grants(nsf_grants_df, db_url):
    """Main function to process and save NSF grants"""
    try:
        # Process grants with embeddings
        processed_df, summary_embeddings, keyword_embeddings = process_grants_with_embeddings(nsf_grants_df)

        # Save to database
        save_to_database(processed_df, summary_embeddings, keyword_embeddings, db_url)

        print("\nProcess completed successfully!")
        return processed_df

    except Exception as e:
        print(f"Error in processing and saving: {e}")
        return None

# Example usage
if __name__ == "__main__":
    # Assuming nsf_grants_df and DB_URL are already defined
    processed_df = process_and_save_nsf_grants(nsf_grants_df, DB_URL)

TimeoutException: Requesting secret DATABASE_URL timed out. Secrets can only be fetched when running from the Colab UI.